In [2]:
import numpy as np
#import cv2

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape, Lambda, Activation, BatchNormalization, LeakyReLU, Dropout
from keras.models import Model
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint 
from keras.utils import plot_model

import tensorflow as tf
import zipfile
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import get_file

In [3]:
# Define the base URL and filename for CelebA dataset
base_url = 'https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip'
file_name = 'celeba.zip'

# Download the CelebA dataset zip file
path = get_file(file_name, base_url, extract=False)

# Load the images from the zip file
with zipfile.ZipFile(path, 'r') as zip_ref:
    zip_ref.extractall('/content/')  # Extract the zip file to a specific directory

# Define the directory path where the images are extracted
data_dir = '/content/img_align_celeba/'

# Load images using TensorFlow's load_img function
images = []
for i in range(1, 202599):  # Assuming the dataset contains 202,599 images
    file_path = data_dir + f'{i:06d}.jpg'  # File naming convention in CelebA dataset
    img = load_img(file_path)
    img = img.resize((64, 64))  # Resize the images to desired size
    img = np.array(img)
    images.append(img)

# Convert the list of images into a numpy array
images = np.array(images)

In [4]:
#Sampling random images from our whole dataset for training and evaluation purporses
import random
images_ds = random.sample(list(images),60000)

In [5]:
images_ds = np.array(images_ds)
type(images_ds)

numpy.ndarray

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(images_ds, test_size=0.2, random_state=42)

In [44]:
type(X_train)

numpy.ndarray

In [8]:
from sklearn import preprocessing as pre

x_train = X_train.reshape(-1, 1)
scaler = pre.MinMaxScaler()
x_train_norm = scaler.fit_transform(x_train)

In [9]:
x_test = X_test.reshape(-1, 1)
x_test_norm = scaler.transform(x_test)

In [25]:
x_train_batch = np.split(X_train,30)
x_test_batch = np.split(X_test,30)

In [32]:
#x_train_batch = np.array(x_train_batch)
x_test_batch = np.array(x_test_batch)

In [33]:
x_train_norm = x_train_batch /255
x_test_norm = x_test_batch /255

In [12]:
# Define input shape and latent dimension
latent_dim = 2
input_shape = (64, 64, 3)
# Encoder network
inputs = Input(shape=input_shape)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
x = Conv2D(32, (3, 3), activation='relu', strides=(2, 2), padding='same')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
shape_before_flattening = K.int_shape(x)
x = Flatten()(x)
z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)

In [13]:
# Sampling function
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim))
    return z_mean + K.exp(z_log_var / 2) * epsilon

# Reparameterization trick
z = Lambda(sampling)([z_mean, z_log_var])

In [14]:
# Decoder network
decoder_input = Input(K.int_shape(z)[1:])
x = Dense(np.prod(shape_before_flattening[1:]), activation='relu')(decoder_input)
x = Reshape(shape_before_flattening[1:])(x)
x = Conv2DTranspose(128, (2, 2), activation='relu', padding='same', )(x)
x = Conv2DTranspose(64, (2, 2), activation='relu', padding='same', strides=(2, 2))(x)
x = Conv2DTranspose(32, (2, 2), activation='relu', padding='same', )(x)
x = Conv2DTranspose(16, (2, 2), activation='relu', padding='same', )(x)
x = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

In [15]:
# Define the VAE model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
decoder = Model(decoder_input, x, name='decoder')
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae')

In [16]:
vae.summary()

Model: "vae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
encoder (Functional)         [(None, 2), (None, 2), (N 1385828   
_________________________________________________________________
decoder (Functional)         (None, 64, 64, 3)         961187    
Total params: 2,347,015
Trainable params: 2,347,015
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Define the VAE loss function
from keras.losses import mse
reconstruction_loss = mse(K.flatten(inputs), K.flatten(outputs))
reconstruction_loss *= input_shape[0] * input_shape[1] * input_shape[2]
kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=1)
B = 1000   
vae_loss = K.mean(B * reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)

In [18]:
vae.add_metric(kl_loss, name="kl_loss")
vae.add_metric(reconstruction_loss, name="reconstruction_loss")

In [40]:
vae.compile(optimizer='adam')
vae.fit(x_train_batch, epochs=30, batch_size=30, validation_data=(x_test_batch, None))

Epoch 1/30


ValueError: in user code:

    C:\Users\DELL\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\DELL\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\DELL\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\DELL\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\DELL\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\DELL\anaconda3\envs\learn